## Assignment 7

### Poojitha Venkatram

#### https://github.com/poojithavenkatram/Deep-Learning

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load the Wikipedia GLoVE Word2Vec

In [3]:
!pip install gensim

In [4]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d glove

--2024-03-28 05:01:23--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-03-28 05:01:23--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-03-28 05:01:23--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [5]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_input_file = 'glove/glove.6B.50d.txt'  # Path to the GLoVE file
word2vec_output_file = 'glove/glove.6B.50d.txt.word2vec'  # Output file
glove2word2vec(glove_input_file, word2vec_output_file)

<ipython-input-5-620fd7923d13>:5: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


(400000, 50)

In [6]:
from gensim.models import KeyedVectors
# Load the converted model
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

### Show how similar are these words

In [7]:
# Check similarity between different pairs of words
words_pairs = [('man', 'woman'), ('chair', 'throne'), ('water', 'baby')]
for word1, word2 in words_pairs:
    print(f"Similarity between {word1} and {word2}: {model.similarity(word1, word2)}")

Similarity between man and woman: 0.8860337734222412
Similarity between chair and throne: 0.27968090772628784
Similarity between water and baby: 0.40810367465019226


### Using these provide analogies for the following

In [8]:
# Define the analogy tasks
analogy_tasks = [
    ('king', 'man', 'woman'),
    ('princess', 'woman', 'man'),
    ('adult', 'child', 'woman')
]

# Solve each analogy
for word1, word2, word3 in analogy_tasks:
    try:
        # Predict the most similar word
        predicted = model.most_similar(positive=[word1, word3], negative=[word2], topn=1)
        print(f"{predicted[0][0]} is to {word1} as {word3} is to {word2}.")
    except KeyError as e:
        print(f"Error: {e}. One of the words in the analogy '{word1}', '{word2}', or '{word3}' is not in the model's vocabulary.")

queen is to king as woman is to man.
prince is to princess as man is to woman.
male is to adult as woman is to child.


### Apply Naive-Bayes Classifier on the Spam-Ham dataset shown in the demo.

In [14]:
import pandas as pd
import string

# Loading the file
file_path = '/content/drive/MyDrive/spam.csv'
df = pd.read_csv(file_path, encoding='latin1')

# Display column names and the first few rows of the DataFrame
print(df.columns)
print(df.head())

# Preprocessing Step 1: Removing punctuations
def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation])

# Preprocessing Step 2: Converting text to lowercase
df['v2'] = df['v2'].apply(lambda x: x.lower())

# Updating the 'label' column
df['v1'] = df['v1'].map({'ham': 0, 'spam': 1})

# Display the updated DataFrame
print(df.head())

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')
     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
   v1                                                 v2 Unnamed: 2  \
0   0  go until jurong point, crazy.. available only ...        NaN   
1   0                      ok lar... joking wif u oni...        NaN   
2   1  free entry in 2 a wkly comp to win fa cup fina...        NaN   
3   0  u dun say so early hor... u c alread

In [15]:
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/spam.csv', encoding='ISO-8859-1')

# Preprocess the data
# For simplicity, we'll only keep the 'v1' (label) and 'v2' (message) columns
df = df[['v1', 'v2']]
df.columns = ['label', 'message']

# Convert labels to numerical format
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

# Vectorize the text messages
vectorizer = CountVectorizer(stop_words='english', lowercase=True)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train the Naive Bayes Classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vectorized, y_train)

# Make predictions
y_pred_train = nb_classifier.predict(X_train_vectorized)
y_pred_test = nb_classifier.predict(X_test_vectorized)

# Evaluate the model
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)
confusion_matrix_test = confusion_matrix(y_test, y_pred_test)
classification_report_test = classification_report(y_test, y_pred_test)

print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")
print("Confusion Matrix:\n", confusion_matrix_test)
print("Classification Report:\n", classification_report_test)

Training Accuracy: 0.9946152120260264
Test Accuracy: 0.9838565022421525
Confusion Matrix:
 [[959   6]
 [ 12 138]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.96      0.92      0.94       150

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115

